# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [105]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [106]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [107]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [108]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [109]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [110]:
np.linalg.matrix_rank(a1)

6

In [111]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/var/folders/js/d6m3lmg13x1dqjyw_1h8pr8h0000gn/T/ipykernel_7233/3327632442.py:15: RuntimeWarning: divide by zero encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]
/var/folders/js/d6m3lmg13x1dqjyw_1h8pr8h0000gn/T/ipykernel_7233/3327632442.py:17: RuntimeWarning: invalid value encountered in matmul
  u = lam @ u
/var/folders/js/d6m3lmg13x1dqjyw_1h8pr8h0000gn/T/ipykernel_7233/3327632442.py:20: RuntimeWarning: invalid value encountered in matmul
  L = L @ lam
/var/folders/js/d6m3lmg13x1dqjyw_1h8pr8h0000gn/T/ipykernel_7233/3327632442.py:15: RuntimeWarning: invalid value encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [92]:
def checking_minor(x):
    shape = x.shape[0] #the function determines the shape of the matrix being tested to decide on the number of iterations in the upcoming for loop
    flag = 0 #the flag variable is set based on the det() value equating to 0
    for i in range(shape+1):
        minor = x[:i,:i]
        det = np.linalg.det(minor)
        if det == 0:
            flag = 1
            break
        else:
            continue
    return 'Success' if flag==0 else 'Fail'

#the print statements below are just calling the above function on a & a1 matrices to check whether any leading minors equal 0.

print(checking_minor(a1)) #since a1 matrix has a leading minor equating to zero our function returns "Fail"
print(checking_minor(a)) #since a matrix has none of the leading minors equal to zero the function returns "Success"

Fail
Success


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [113]:
def diy_lu_pivot(a):
    
    n = len(a)
    c = a
    p = np.eye(n)
    for i in range(n):
      pivotVal = 0
      pivot = -1
      for row in range(i, n):
        if math.fabs(c[row][i]) > pivotVal:
          pivotVal = math.fabs(c[row][i])
          pivot = row
          
      if pivotVal != 0:
        p[i], p[pivot] = p[pivot], p[i]
        c[i], c[pivot] = c[pivot], c[i]
        for j in range(i+1, n):
          c[j][i] /= c[i][i]
          for k in range(i+1, n):
            c[j][k] -= c[j][i]*c[i][k]  
    return c

In [122]:
import math
final = diy_lu_pivot(a1)
print(final)


[[ 3.     3.     3.     3.     3.     3.   ]
 [ 0.333 -3.25  -3.571 -3.7   -3.769 -3.812]
 [ 0.333 -0.    -1.    -1.    -1.    -1.   ]
 [ 0.333  0.044  0.262 -0.569 -0.56  -0.554]
 [ 0.333  0.018  0.682 -0.754 -0.672 -0.667]
 [ 0.333 -0.     1.    -0.    -0.     0.   ]]
